---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def degree_distribution(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
    return histogram

In [6]:
for i, G in enumerate(P1_Graphs):
    print(i, nx.average_clustering(G), nx.average_shortest_path_length(G), len(degree_distribution(G)))

0 0.03167539146454044 4.099161161161161 29
1 0.5642419635919628 5.089871871871872 7
2 0.4018222222222227 9.378702269692925 5
3 0.03780379975223251 3.1048046283934134 37
4 0.0033037037037037037 5.0785509568313305 9


In [7]:
def graph_identification():   
    return list(['PA', 'SW_L', 'SW_L', 'PA', 'SW_H'])

In [8]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [9]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [10]:
G.nodes(data=True)[:10]

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0}),
 (5, {'Department': 25, 'ManagementSalary': nan}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (7, {'Department': 14, 'ManagementSalary': 0.0}),
 (8, {'Department': 14, 'ManagementSalary': nan}),
 (9, {'Department': 14, 'ManagementSalary': 0.0})]

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def salary_predictions():
    df = pd.DataFrame(index=G.nodes())
    
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['page_rank'] = pd.Series(nx.pagerank(G))
    df['department'] = pd.Series([node[1]['Department'] for node in G.nodes(data=True)])
    df['target'] = pd.Series([node[1]['ManagementSalary'] for node in G.nodes(data=True)])

    df_train = df[~pd.isnull(df['target'])]
    df_test  = df[ pd.isnull(df['target'])]

    X_train = df_train.drop('target', axis=1)
    y_train = df_train['target'].astype(int)
    X_test = df_test.drop('target', axis=1)

    clf = RandomForestClassifier()
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')
    print(scores)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict_proba(X_test)[:, 1]
    
    return pd.Series(y_pred, X_test.index)

In [12]:
salary_predictions().head()

[ 0.9312664   0.9539042   0.95997375  0.93848425  0.86818496]


1     0.0
2     1.0
5     1.0
8     0.4
14    0.0
dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [13]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Future Connection
(6, 840)                  0.0
(4, 197)                  0.0
(620, 979)                0.0
(519, 872)                0.0
(382, 423)                0.0
(97, 226)                 1.0
(349, 905)                0.0
(429, 860)                0.0
(309, 989)                0.0
(468, 880)                0.0

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [14]:
def new_connections_predictions():
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    df['cn_soundarajan_hopcroft'] = [x[2] for x in list(nx.cn_soundarajan_hopcroft(G))]
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    
    df = future_connections.join(df, how='outer')
    
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test  = df[ pd.isnull(df['Future Connection'])]
    
    X_train = df_train.drop('Future Connection', axis=1)
    y_train = df_train['Future Connection'].astype(int)
    X_test = df_test.drop('Future Connection', axis=1)

    clf = RandomForestClassifier()
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')
    print(scores)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict_proba(X_test)[:, 1]
    
    return pd.Series(y_pred, X_test.index)

In [15]:
new_connections_predictions().head()

[ 0.90786813  0.88376108  0.8718486   0.85038204  0.74793888]


(0, 9)     0.0
(0, 19)    0.0
(0, 20)    0.2
(0, 35)    0.0
(0, 38)    0.0
dtype: float64